<a href="https://colab.research.google.com/github/HardikPrabhu/Natural-Language-Processing-on-the-Covid-19-Research-papers/blob/main/Hardik_Prabhu_NlpAssignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hardik Prabhu

Roll Number: MDS201912

**Picking up from where we left...**

In the previous notebook, we preprocessed the json corpus such that each of the document got converted into bag of bag of words (list of list). We also decided on retaining the stopwords as it will help us to make a better n-gram model, which would  mimic the syntatical struture of the sentences. This lead to blow up of the memory usage and we had to resort to partitioning of the corpus into 16 parts. Each partion was preprocessed seperatly and then after preprocessing, each stored as a pickle objects along with their respective vocabularies (list of words).

We will use the preprocesssed corpus to create n-gram models.



**Fetching the pickle objects**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
location="/content/drive/MyDrive/pdf_json"  #location of json files
save_location="/content/drive/MyDrive"      #location on my gdrive
os.chdir(save_location)


Getting the vocabulary..

In [ ]:
import pickle

In [ ]:
vocabulary=set()  #entire vocabulary by taking the union over the partitons. 
os.chdir(save_location)
for m in range(16):
  with open("vocab-"+str(m), "rb") as f:
    
     vocab=pickle.load(f)
         
     
  vocabulary=vocabulary.union(vocab)

print("Vocabulary size: "+ str(len(vocabulary)))

Vocabulary size: 801851


**Converting into n-tuples**

The sentences(list of words) are processed further to get a collection of n-tuples which are required for creation of the models.

In [ ]:
#tuple extraction
from nltk.util import ngrams
def n_tuple_extract(Corpus,n):
  c=[]
  for doc in Corpus:
    for sentence in doc:
      x=list(ngrams(sentence,n, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
      c=c+x
  return c

**Tri-gram model**

In [ ]:
#example
corp=[[["this", "is", "a", "sentence"],["another", "sentence", "in","the","same", "document"]],[["a","sentence", "in","different","document"],["this","is","a","short","document"]]]
print(n_tuple_extract(corp,3))

[('<s>', '<s>', 'this'), ('<s>', 'this', 'is'), ('this', 'is', 'a'), ('is', 'a', 'sentence'), ('a', 'sentence', '</s>'), ('sentence', '</s>', '</s>'), ('<s>', '<s>', 'another'), ('<s>', 'another', 'sentence'), ('another', 'sentence', 'in'), ('sentence', 'in', 'the'), ('in', 'the', 'same'), ('the', 'same', 'document'), ('same', 'document', '</s>'), ('document', '</s>', '</s>'), ('<s>', '<s>', 'a'), ('<s>', 'a', 'sentence'), ('a', 'sentence', 'in'), ('sentence', 'in', 'different'), ('in', 'different', 'document'), ('different', 'document', '</s>'), ('document', '</s>', '</s>'), ('<s>', '<s>', 'this'), ('<s>', 'this', 'is'), ('this', 'is', 'a'), ('is', 'a', 'short'), ('a', 'short', 'document'), ('short', 'document', '</s>'), ('document', '</s>', '</s>')]


We maintain a dictionary of counts in order to generate probabilities.

In [ ]:

def counting(c,counts):
 
 for i in c:
    if (i[:-1],i[-1]) in counts:
        counts[i[:-1],i[-1]]=counts[i[:-1],i[-1]]+1
    else:
        counts[i[:-1], i[-1]]=1
 return counts                        
        

In [ ]:
counts={}
counting(n_tuple_extract(corp,3),counts)
print(counts)

{(('<s>', '<s>'), 'this'): 2, (('<s>', 'this'), 'is'): 2, (('this', 'is'), 'a'): 2, (('is', 'a'), 'sentence'): 1, (('a', 'sentence'), '</s>'): 1, (('sentence', '</s>'), '</s>'): 1, (('<s>', '<s>'), 'another'): 1, (('<s>', 'another'), 'sentence'): 1, (('another', 'sentence'), 'in'): 1, (('sentence', 'in'), 'the'): 1, (('in', 'the'), 'same'): 1, (('the', 'same'), 'document'): 1, (('same', 'document'), '</s>'): 1, (('document', '</s>'), '</s>'): 3, (('<s>', '<s>'), 'a'): 1, (('<s>', 'a'), 'sentence'): 1, (('a', 'sentence'), 'in'): 1, (('sentence', 'in'), 'different'): 1, (('in', 'different'), 'document'): 1, (('different', 'document'), '</s>'): 1, (('is', 'a'), 'short'): 1, (('a', 'short'), 'document'): 1, (('short', 'document'), '</s>'): 1}


We argue that maintaing such a dictionary will yield sufficient information to generate probabilities.
Hence, once counts are generated, the tuples can be thrown away to save ram blow up.

In [ ]:
counts={}
for m in range(15):
  
  with open("nlp-"+str(m), "rb") as f:
    corp=pickle.load(f)
    print("corpus"+str(m)+" loaded")
  x=n_tuple_extract(corp,3)
  print("trigrams created")
  counting(x,counts)
  x=[]
  print("partion done") 

with open("trigram counts-1","wb") as f: #save it for later
  pickle.dump(counts,f)



                              
  


corpus0 loaded


KeyboardInterrupt: ignored

I have been struggling for weeks to resolve the memory blow up. The above stategy of dividing into 16 parts failed  as well. Google allows only 15 gb storage max and 12 gb ram. Its not enough to work with the entire corpus.
A lot of efforts into approaching part by  part has resulted in a lot of runtime error. I will be working on a seperate colab notebook. Please accept the submission done on the smaller portion of the data. 


I will use a small section (1000 documents) of the entire corpus for trainning.

In [ ]:
with open("nlp-1", "rb") as f:
    corp=pickle.load(f)

In [ ]:
with open("vocab-1","rb") as f: #vocabulary for the section of corpus
  vocab=pickle.load(f)

In [ ]:
test=corp[1000:1004] #for evaluationg the model later
corp=corp[:1000]

In [ ]:
counts={}
x=n_tuple_extract(corp,3)
print("trigrams created")
counting(x,counts)

with open("trigram counts","wb") as f: #save it for later
  pickle.dump(counts,f)


trigrams created


In [ ]:
with open("trigram counts","rb") as f: 
  counts=pickle.load(f)

len(counts)

2472276

In [ ]:
#generating probabilities
def prob(x):
    total=0
    z=0
    if (x[:-1],x[-1]) in counts:
     z=counts[x[:-1],x[-1]]

     for y in counts:
        if y[0]==x[:-1]:
            total=total+counts[y]

     return z/total
    else:
        return 0


In [ ]:
#generating probability for p(serious| poses a):
prob(('poses', 'a', 'serious'))

0.1111111111111111

We will use the function prob to generate a random sentence.

In [ ]:
import numpy
def generate(s):
    y=s.split()
    probability=[]
    words=[]
    for v in vocab:
      z=prob(tuple(y[-2:]+[v]))
      if z!=0:
         probability.append(z)
         words.append(v)
    try:
     word= numpy.random.choice(numpy.arange(0, len(words)), p=probability)
     s=s+" "+ words[word]
    except: #due to some numeric error probabilities may not sum up to 1 (valueerror)
        p=1-sum(probability)
        words.append("</s> </s> <s> <s>")
        probability.append(p)
        word = numpy.random.choice(numpy.arange(0, len(words)), p=probability)
        s = s + " " + words[word]

    return s


We have created a function which would given a string as an input will generate a new word and add it at the end and return the string. 

The function simply looks at the last 2 words (w1,w2) in the string and for any word w in the vocabulary calculates p(w| w1, w2) using the prob fuction which was defined earlier. We maintain a list on words with non-zero probability and a list of respective probabilities and use the probabilities to sample a word from the list.   

Ideally we would like to see the sum of the probabilities to be 1 over the entire vocabulary. But since we are adding a large number of small floating point numbers, they may not add up to 1 and a value error might be thrown.
To tackle this we give remaing probability to the seq $</s> </s> <s> <s>$  indicating end of the old sentence and beginning of a new one. 



In [ ]:
#creating a 20 words string using trigarm model
s="<s> <s>"
for i in range(20):
  s=generate(s)

print(s)

<s> <s> negative interruptions are one of the role of ezrin in hepatocytes c regulates the transport sector is weak eg social


In [ ]:
# most occuring starting word
max=0
word=""
for j in vocab:
  try:
   c=counts[("<s>","<s>"),j]
   if c>max:
    max=c
    word=j
  except:
    pass


In [ ]:
print (word)

the


The word with max occurence at the beginning is: "the"

In [ ]:
s="<s> the"
for i in range(25):
  s=generate(s)

print(s)

<s> the role of host factors such as ssagwtagaa of microbacteria cover match or outperform other outcome predictors such as temperature conductivity and s in ii and


**The 4-gram model**

In [ ]:
#example 
c=[[["this", "is", "a", "sentence"],["another", "sentence", "in","the","same", "document"]],[["a","sentence", "in","different","document"],["this","is","a","short","document"]]]
print(n_tuple_extract(c,4))

[('<s>', '<s>', '<s>', 'this'), ('<s>', '<s>', 'this', 'is'), ('<s>', 'this', 'is', 'a'), ('this', 'is', 'a', 'sentence'), ('is', 'a', 'sentence', '</s>'), ('a', 'sentence', '</s>', '</s>'), ('sentence', '</s>', '</s>', '</s>'), ('<s>', '<s>', '<s>', 'another'), ('<s>', '<s>', 'another', 'sentence'), ('<s>', 'another', 'sentence', 'in'), ('another', 'sentence', 'in', 'the'), ('sentence', 'in', 'the', 'same'), ('in', 'the', 'same', 'document'), ('the', 'same', 'document', '</s>'), ('same', 'document', '</s>', '</s>'), ('document', '</s>', '</s>', '</s>'), ('<s>', '<s>', '<s>', 'a'), ('<s>', '<s>', 'a', 'sentence'), ('<s>', 'a', 'sentence', 'in'), ('a', 'sentence', 'in', 'different'), ('sentence', 'in', 'different', 'document'), ('in', 'different', 'document', '</s>'), ('different', 'document', '</s>', '</s>'), ('document', '</s>', '</s>', '</s>'), ('<s>', '<s>', '<s>', 'this'), ('<s>', '<s>', 'this', 'is'), ('<s>', 'this', 'is', 'a'), ('this', 'is', 'a', 'short'), ('is', 'a', 'short', '

Using the same counting fuction as before to generate a dictionary of counts.

In [ ]:
counts4={}
counting(n_tuple_extract(c,4),counts4)


{(('<s>', '<s>', '<s>'), 'a'): 1,
 (('<s>', '<s>', '<s>'), 'another'): 1,
 (('<s>', '<s>', '<s>'), 'this'): 2,
 (('<s>', '<s>', 'a'), 'sentence'): 1,
 (('<s>', '<s>', 'another'), 'sentence'): 1,
 (('<s>', '<s>', 'this'), 'is'): 2,
 (('<s>', 'a', 'sentence'), 'in'): 1,
 (('<s>', 'another', 'sentence'), 'in'): 1,
 (('<s>', 'this', 'is'), 'a'): 2,
 (('a', 'sentence', '</s>'), '</s>'): 1,
 (('a', 'sentence', 'in'), 'different'): 1,
 (('a', 'short', 'document'), '</s>'): 1,
 (('another', 'sentence', 'in'), 'the'): 1,
 (('different', 'document', '</s>'), '</s>'): 1,
 (('document', '</s>', '</s>'), '</s>'): 3,
 (('in', 'different', 'document'), '</s>'): 1,
 (('in', 'the', 'same'), 'document'): 1,
 (('is', 'a', 'sentence'), '</s>'): 1,
 (('is', 'a', 'short'), 'document'): 1,
 (('same', 'document', '</s>'), '</s>'): 1,
 (('sentence', '</s>', '</s>'), '</s>'): 1,
 (('sentence', 'in', 'different'), 'document'): 1,
 (('sentence', 'in', 'the'), 'same'): 1,
 (('short', 'document', '</s>'), '</s>'): 

In [ ]:
counts4={}
x=n_tuple_extract(corp,4)
print("4-grams created")
counting(x,counts4)

with open("4-gram counts","wb") as f: #save it for later
  pickle.dump(counts4,f)

4-grams created


In [ ]:
with open("4-gram counts","rb") as f: 
  counts4=pickle.load(f)

In [ ]:
len(counts4)

3262901

In [ ]:
def prob4(x):
    total=0
    z=0
    if (x[:-1],x[-1]) in counts4:
     z=counts4[x[:-1],x[-1]]

     for y in counts4:
        if y[0]==x[:-1]:
            total=total+counts4[y]

     return z/total
    else:
        return 0




In [ ]:
#example
prob4(("<s>","<s>","<s>","the"))

0.13517611219519043

In [ ]:
def generate4(s):
    y=s.split()
    probability=[]
    words=[]
    for v in vocab:
      z=prob4(tuple(y[-3:]+[v]))
      if z!=0:
         probability.append(z)
         words.append(v)
    try:
     word= numpy.random.choice(numpy.arange(0, len(words)), p=probability)
     s=s+" "+ words[word]
    except: #due to some numeric error probabilities may not sum up to 1 (valueerror)
        p=1-sum(probability)
        words.append("</s> </s> </s> <s> <s> <s>")
        probability.append(p)
        word = numpy.random.choice(numpy.arange(0, len(words)), p=probability)
        s = s + " " + words[word]

    return s

In [ ]:
#the starting word with maximal occurence
max=0
word=""
for j in vocab:
  try:
   c=counts4[("<s>","<s>","<s>"),j]
   if c>max:
    max=c
    word=j
  except:
    pass


In [ ]:
print(word)

the


In [ ]:
s="<s> <s> the"
for i in range(25):
  s=generate4(s)

print(s)

<s> <s> the pharmaceutical in nanjangud is in the process of enhancing hot cognitions by discussing food intake just before weighing to enhance excessive weight predictions and consequent


**Comparing the two models:**


We will evaluate our language models by comparing log of perplexity obtained over a test dataset. Perpelxity is a measure of fit, lower the better. 

In [ ]:
def log_perplexity_trigram(trigrams,eps): #small amount eps is added to handle when prob(tuple) is zero 
     perp=0
     for tuple in trigrams:
         perp=perp+numpy.log(prob(tuple)+eps)  #applying the chain rule p(w1,w2,w3,..wn)=p(w1,w2)p(w3|w2,w1)...p(wn|wn-1,wn-2)
     #p(w1,w2) i.e p(<s> <s>): counts of trigrams statring with <s> <s> / total number of trigrams in tranning corpus
     total=0
     count=0
     for j in counts:
         total=total +counts[j]
         if j[0]==("<s>","<s>"):
             count=count+counts[j]
     p=count/total
     perp=perp+numpy.log(p)
     perp=-1*perp/len(trigrams)
     return perp


In [ ]:
def log_perplexity_4gram(fgrams,eps):
     perp=0
     for tuple in fgrams:
         perp=perp+numpy.log(prob4(tuple)+eps)  #applying the chain rule p(w1,w2,w3,..wn)=p(w1,w2, w3)p(w4|w3,w2,w1)...p(wn|wn-1,wn-2,wn-3)
     #p(w1,w2,w3) i.e p(<s> <s> <s>): counts of 4-grams statring with <s> <s> <s> / total number of 4-grams in tranning corpus
     total=0
     count=0
     for j in counts4:
         total=total +counts4[j]
         if j[0]==("<s>","<s>","s"):
             count=count+counts4[j]
     p=count/total
     perp=perp+numpy.log(p)
     perp=-1*perp/len(fgrams)
     return perp


In [ ]:
test1=n_tuple_extract(test,3)
test2=n_tuple_extract(test,4)

In [ ]:
#perplexity calculations over the test corpus
print("The trigram model")
print(log_perplexity_trigram(test1,0.000001))
print("The 4-gram model")
print(log_perplexity_4gram(test2,0.000001))

The trigram model
10.506219706915168
The 4-gram model
12.230797815587836


It seems that the tri-gram model is fitting better than the 4-gram model on the test corpus. (has a lower log perplexity)


Q.6. Ans: The only model parameters are the dictionaries of counts. we use the dictionaries to generate probablities. To handle a large dictionary, we can maintain multiple small dictionaries instead of a large one and can sweep across them in parallel  and add up the counts while generating probabilities.

Q.7   Ans: We can partition the corpus and then on each partition create n-grams parallelly. We can maintain separate dictionaries of counts for each of the partitions and then add up the counts from each partition while generating probabilities.  